In [1]:
using Erdos
using Distributions
using Gadfly
using Clustering
using MatrixNetworks

g = readgraph(:emailenron);
a = adjacency_list(g::AGraph);

N = size(a)[1];
A0  = spzeros(N,N);

A0 = sparse(1:N,1:N,0*vec(ones(Int64,N,1)),N,N);

for i=1:N
    l = a[i];
    for j=1:length(l)
        A0[i,l[j]]=1;
        A0[l[j],i]=1;
    end
end
A,p = largest_component(A0);
A0=0;p=0;a=0;g=0;
gc();

A0 =A;
l = size(A,1);
for i =1:100
    d = sum(A0,2);
    for i=1:length(d)
        if d[i]==1
            for j=1:length(d)
                if A0[i,j] == 1
                    A0[i,j] = 0;
                    A0[j,i] = 0;
                end
            end
        end
    end
    A0 = sparse(A0);dropzeros!(A0);

    A0,p = largest_component(A0);

    if l == size(A0,1)
        break
    else
        l = size(A0,1);
        print(size(A0));print("\n");

    end
end

A = A0;
A0 =0;
gc()

(24232, 24232)
(24155, 24155)
(24147, 24147)
(24146, 24146)
(24145, 24145)
(24144, 24144)


In [2]:
d = sum(A,2);
s = sum(d);

r = 100;
N = length(d); 

v0 = rand(Uniform(-1,1),N,r);
v0 = v0./sqrt.(sum(abs2, v0, 1));
v0, t = qr(v0);
l0 = diag(t);


### Preprocess ###
u0 = rand(Uniform(-1,1),N,1);
u0 = u0./norm(u0);
n=0;
for it = 1:30
    u0 = A*u0 .- (d/s)*(d'*u0);
    n = norm(u0,2);
    u0 = u0/n;
end
shift = n;

l = l0;
##################

it_max=10000;
it = 1;
while it <=it_max
    v0 = A*v0 .- (d/s)*(d'*v0) .+ shift*v0;
    v0, t = qr(v0);
    l = diag(t);
    if norm(vec(abs.(l))-vec(abs.(l0)))<1e-03
             break;
    end
    l0 = l;
    it = it+1;
end

if it ==it_max
        print("No convergence")
end 

print(l0-shift)

[74.808, 70.3629, 63.775, 62.3993, 54.2385, 49.8959, 47.3853, 46.5607, 43.1513, 39.8839, 39.6777, 38.9105, 36.7373, 36.6136, 35.4765, 34.593, 33.9837, 32.9679, 31.9607, 31.5468, 30.2897, 29.4464, 29.2444, 28.7357, 28.2066, 28.1252, 27.6857, 27.0684, 26.5763, 26.4138, 25.8381, 25.9562, 25.6991, 25.1257, 24.8596, 24.7076, 24.6266, 24.4047, 24.078, 24.1636, 23.8053, 23.6781, 23.6077, 23.3891, 22.9405, 22.7266, 22.7247, 22.5072, 22.3835, 21.9441, 22.0644, 21.7693, 21.6146, 21.5102, 21.3793, 21.257, 21.0927, 21.0197, 20.6724, 20.6274, 20.4732, 20.4244, 20.3425, 20.0181, 20.2306, 20.0295, 19.9685, 19.8301, 19.6963, 19.5635, 19.6438, 19.5736, 19.4605, 19.3232, 19.381, 19.3387, 18.9817, 19.0463, 18.8353, 18.8545, 18.7415, 18.7922, 18.5408, 18.5954, 18.4565, 18.4426, 18.3101, 18.0698, 18.1274, 17.9017, 17.9412, 17.835, 17.8178, 17.5546, 17.546, 17.5732, 17.5704, 17.4028, 17.3661, 17.2628]

In [8]:
include("Algo/Partition.jl")
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")

Reorder (generic function with 1 method)

In [24]:
Sig = v0[:,1:10]';

n_clusters = 200;
n_it_Kmeans = 1000;
n_updates = 100;


community = vec(zeros(N,1));
community0 = vec(zeros(N,1));
Q_best = 0;
n_c_best = 0;
n_c = 0;
# Probability mass for sampling the initial centroids
p = vec(d/s);
n_updates_best = 0;

for k =1:n_it_Kmeans

	community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
	n_c = length(unique(community0));

    	if Q>Q_best
		community = community0;
		Q_best = Q;
		n_c_best = n_c;
		n_updates_best = n_updates;

		
    	end
end	
  
	community = Reorder(community);

	print("Number of updates: ")
	print(n_updates_best)
	print("\n")
	print("Number of communities: ")
	print(n_c_best)
	print("\n")

	print("Modularity: ")
	print(Q_best)
	print("\n")

Number of updates: 2
Number of communities: 13
Modularity: 0.5259354080623625


In [3]:
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")
include("Algo/Partition.jl")

n_clusters = 200;
n_it_Kmeans = 100;
n_updates = 100;
community = vec(zeros(N,1));
community0 = vec(zeros(N,1));

Q_best_best = 0;
n_c_best_best = 0;
n_updates_best_best = 0;
r_best = 2;
for r=2:80
    
    Sig = v0[:,1:r]';
    Q_best = 0;
    n_c_best = 0;
    n_c = 0;
    # Probability mass for sampling the initial centroids
    p = vec(d/s);
    n_updates_best = 0;

    for k =1:n_it_Kmeans

        community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
        n_c = length(unique(community0));
            if Q>Q_best
            community = community0;
            Q_best = Q;
            n_c_best = n_c;
            n_updates_best = n_updates;
            end
    end	

    community = Reorder(community);
    
    if Q_best>Q_best_best 
        r_best = r;
        Q_best_best = Q_best;
        n_c_best_best = n_c_best;
        n_updates_best_best = n_updates_best;
    end
    
end


	print("Number of updates: ")
	print(n_updates_best_best )
	print("\n")
	print("Number of communities: ")
	print(n_c_best_best )
	print("\n")

	print("Modularity: ")
	print(Q_best_best )
	print("\n")
	print("n: ")
	print(r_best )
	print("\n")


Number of updates: 1
Number of communities: 12
Modularity: 0.5230858447359333
n: 7
